# EV Analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
set2_colors = cm.get_cmap('Set2')
color1 = set2_colors(0)
color2 = set2_colors(1)

C:\Users\Ricky Chen\AppData\Local\Temp\ipykernel_12364\1779802014.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  set2_colors = cm.get_cmap('Set2')


## 1 Weight adjustment

In [2]:
def weight_adjust(EF, weight, a, weight_ref=1406):
    """
    Adjust calculated EF according to weight.
    """
    return EF * (weight/weight_ref)**(1/a)

## 2 Sensitivity analysis

OpMode ERs data.

In [3]:
OpModeERs = np.load("./data/agg/OpModeERs.npy")
OpModeERs_Ei5 = np.load("./data/agg/OpModeERs_Ei5.npy")
OpModeERs_ModelY = np.load("./data/agg/OpModeERs_ModelY.npy")

Link aggregation data.

In [4]:
import pickle
with open("./data/agg/agg_link.pkl", 'rb') as f:
    agg_link = pickle.load(f)

import geopandas as gpd
roadnet = gpd.read_file("./data/geo/roadnet/roadnet_simplified.shp")

# link-level aggregation
agg_link = agg_link.join(roadnet.set_index('osmid'))
agg_link = gpd.GeoDataFrame(agg_link, geometry='geometry')

# parameter calculation
agg_link.dropna(inplace=True)
agg_link['brakeFrac'] = agg_link['brakeCount'] / agg_link['trajCount']
agg_link['brakeFreq'] = agg_link['brakeEventNum'] / agg_link['mileage']  # #/km
agg_link['brakeTimeMean'] = agg_link['brakeCount'] / agg_link['brakeEventNum']  # sec
agg_link['OpModeFrac'] = agg_link['OpModeCount'] / agg_link['trajCount']
agg_link['brakeDecelMean'] = agg_link['brakeDecelMean'].apply(np.abs)
agg_link.replace(np.inf, 0, inplace=True)
agg_link.fillna(0, inplace=True)

In [5]:
agg_link['ER'] = agg_link['OpModeFrac'].apply(lambda x: sum(x * OpModeERs))
agg_link['EF'] = agg_link['ER'] * agg_link['trajCount'] / 3600 / agg_link['mileage'] * 1000  # mg/km/veh

agg_link['ER_Ei5'] = agg_link['OpModeFrac'].apply(lambda x: sum(x * OpModeERs_Ei5))
agg_link['EF_Ei5'] = agg_link['ER_Ei5'] * agg_link['trajCount'] / 3600 / agg_link['mileage'] * 1000  # mg/km/veh

agg_link['ER_ModelY'] = agg_link['OpModeFrac'].apply(lambda x: sum(x * OpModeERs_ModelY))
agg_link['EF_ModelY'] = agg_link['ER_ModelY'] * agg_link['trajCount'] / 3600 / agg_link['mileage'] * 1000  # mg/km/veh

In [6]:
agg_link = agg_link[['trajCount', 'EF', 'EF_Ei5', 'EF_ModelY', 'road type', 'geometry']]

### 2.1 EV emission factor

In [7]:
def cal_EF(weight, reg, a):
    # select regenerative braking level
    if reg == 'zero':
        EF_key = 'EF'
    elif reg == 'weak':
        EF_key = 'EF_Ei5'
    else:
        EF_key = 'EF_ModelY'
    # calculate average EF and std
    avg_EF = sum(agg_link[EF_key] * agg_link['trajCount']) / agg_link['trajCount'].sum()
    EF_std = agg_link.apply(lambda x: x['trajCount']*np.power((x[EF_key]-avg_EF), 2), axis=1).sum()
    EF_std /= agg_link['trajCount'].sum()
    EF_std = np.sqrt(EF_std)
    # weight adjust
    return weight_adjust(avg_EF, weight, a), weight_adjust(EF_std, weight, a)

In [8]:
columns = ['weight', 'reg', 'a', 'EF', 'std']
EV_EFs = pd.DataFrame({}, columns=columns)

# calcualte weight-dependent EF
for weight in range(1400, 3500, 500):
    for reg in ['zero', 'weak', 'strong']:
        for a in [1, 1.9]:
            EF, std = cal_EF(weight, reg, a)
            EV_EFs.loc[EV_EFs.shape[0]] =[weight, reg, a, EF, std]

In [9]:
# EV_EFs.to_csv("data/ev/EV_EFs.csv", index=False)

### Reg reduction

In [10]:
linear = EV_EFs[EV_EFs['a']==1]
zero = linear[linear['reg'] == 'zero']
weak = linear[linear['reg'] == 'weak']
strong = linear[linear['reg'] == 'strong']

pd.DataFrame({
    'weight': zero['weight'].values,
    'weak_reduce': (zero['EF'].values - weak['EF'].values) / zero['EF'].values,
    'strong_reduce': (zero['EF'].values - strong['EF'].values) / zero['EF'].values,
})

,weight,weak_reduce,strong_reduce
0,1400,0.722604,0.926529
1,1900,0.722604,0.926529
2,2400,0.722604,0.926529
3,2900,0.722604,0.926529
4,3400,0.722604,0.926529


In [11]:
exp = EV_EFs[EV_EFs['a']==1.9]
zero = exp[exp['reg'] == 'zero']
weak = exp[exp['reg'] == 'weak']
strong = exp[exp['reg'] == 'strong']

pd.DataFrame({
    'weight': zero['weight'].values,
    'weak_reduce': (zero['EF'].values - weak['EF'].values) / zero['EF'].values,
    'strong_reduce': (zero['EF'].values - strong['EF'].values) / zero['EF'].values,
})

,weight,weak_reduce,strong_reduce
0,1400,0.722604,0.926529
1,1900,0.722604,0.926529
2,2400,0.722604,0.926529
3,2900,0.722604,0.926529
4,3400,0.722604,0.926529


In [12]:
linear = EV_EFs[EV_EFs['a']==1]
zero = linear[linear['reg'] == 'zero']
weak = linear[linear['reg'] == 'weak']
strong = linear[linear['reg'] == 'strong']

pd.DataFrame({
    'weight': zero['weight'].values,
    'zero_EF_increase': (zero['EF'].shift(-1) - zero['EF']).values,
    'weak_EF': (weak['EF'].shift(-1) - weak['EF']).values,
    'strong_EF': (strong['EF'].shift(-1) - strong['EF']).values,
})

,weight,zero_EF_increase,weak_EF,strong_EF
0,1400,9.547596,2.648469,0.701473
1,1900,9.547596,2.648469,0.701473
2,2400,9.547596,2.648469,0.701473
3,2900,9.547596,2.648469,0.701473
4,3400,NaN,NaN,NaN


In [13]:
exp = EV_EFs[EV_EFs['a']==1.9]
zero = exp[exp['reg'] == 'zero']
weak = exp[exp['reg'] == 'weak']
strong = exp[exp['reg'] == 'strong']

pd.DataFrame({
    'weight': zero['weight'].values,
    'zero_EF_increase': (zero['EF'].shift(-1) - zero['EF']).values,
    'weak_EF': (weak['EF'].shift(-1) - weak['EF']).values,
    'strong_EF': (strong['EF'].shift(-1) - strong['EF']).values,
}).describe()

,weight,zero_EF_increase,weak_EF,strong_EF
count,5.000000,4.000000,4.000000,4.000000
mean,2400.000000,3.986004,1.105703,0.292856
std,790.569415,0.535643,0.148585,0.039354
min,1400.000000,3.431752,0.951956,0.252135
25%,1900.000000,3.652195,1.013106,0.268331
50%,2400.000000,3.920739,1.087599,0.288061
75%,2900.000000,4.254548,1.180196,0.312586
max,3400.000000,4.670786,1.295659,0.343168


### 2.2 Trends

In [9]:
penetration = pd.read_excel("data/ev/penetration.xlsx")
penetration.set_index('year', inplace=True)
penetration

,population,ev population,penetration,bev%
year,,,,
2016,184000000,1014000,0.005511,NaN
2017,217000000,1530000,0.007051,NaN
2018,231000000,2610000,0.011299,NaN
2019,260000000,3810000,0.014654,0.812
2020,281000000,4920000,0.017509,0.810
2021,302000000,7832160,0.025934,0.816
2022,319000000,13100000,0.041066,0.798
2023,336000000,20410000,0.060744,0.760


In [15]:
# import matplotlib.ticker as ticker

# fig, ax1 = plt.subplots(figsize=(7,3))
# ax2 = ax1.twinx()

# x = np.arange(len(penetration['year']))
# width = 0.35
# ax1.bar(x, penetration['population'], width, label='all vehicles', color=color1)
# ax1.bar(x, penetration['ev population'], width, label='electric vehicles', color=color2)
# ax1.set_xticks(x, penetration['year'])
# ax1.set_ylabel('Population')

# line, = ax2.plot(x, penetration['penetration'], 'D-', color='darkslategrey', label='EV Penetration')
# ax2.set_ylabel('Penetration')
# for i, v in enumerate(penetration['penetration']):
#     ax2.text(i, v, f'{v * 100:.1f}%', color='darkslategrey', ha='right', va='bottom')

# lines, labels = ax1.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# ax2.legend(lines + lines2, labels + labels2, loc='upper left')
# ax2.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=1))

# ax1.spines['top'].set_visible(False)
# ax2.spines['top'].set_visible(False)
# plt.show()

In [10]:
weight = pd.read_csv("data/ev/weights_2016_2023.csv")
weight['inc_rate'] = (weight['ev'] - weight['icev']) / weight['icev']
weight.set_index('year', inplace=True)
weight

,icev,bev,hev,ev,inc_rate
year,,,,,
2016.0,1407.156133,1443.526639,1544.126185,1463.646548,0.040145
2017.0,1407.096447,1447.944149,1544.109606,1467.177240,0.042698
2018.0,1408.790029,1532.115814,1548.185079,1535.329667,0.089822
2019.0,1412.003449,1574.090186,1565.999669,1572.472082,0.113646
2020.0,1419.234621,1639.755298,1583.427437,1628.489726,0.147442
2021.0,1424.845006,1724.998029,1634.103609,1706.819145,0.197898
2022.0,1430.748765,1741.890237,1731.946080,1739.901406,0.216078
2023.0,1438.749570,1735.703284,1804.681774,1749.498982,0.215986


### vehicle-level EF

In [22]:
agg_link_2016 = agg_link.copy()
agg_link_2017 = agg_link.copy()
agg_link_2018 = agg_link.copy()
agg_link_2019 = agg_link.copy()
agg_link_2020 = agg_link.copy()
agg_link_2021 = agg_link.copy()
agg_link_2022 = agg_link.copy()
agg_link_2023 = agg_link.copy()
agg_links = {
    2016: agg_link_2016,
    2017: agg_link_2017,
    2018: agg_link_2018,
    2019: agg_link_2019,
    2020: agg_link_2020,
    2021: agg_link_2021,
    2022: agg_link_2022,
    2023: agg_link_2023,
}

In [23]:
# calculate vehicle-level EFs after weight-adjustment
for year, agg in agg_links.items():
    # linear assumption
    agg['EF_linear'] = agg['EF'].apply(lambda x: weight_adjust(x, weight.loc[year]['icev'], 1))
    agg['EF_Ei5_linear'] = agg['EF_Ei5'].apply(lambda x: weight_adjust(x, weight.loc[year]['ev'], 1))
    agg['EF_ModelY_linear'] = agg['EF_ModelY'].apply(lambda x: weight_adjust(x, weight.loc[year]['ev'], 1))
    # exponetial assumption
    agg['EF_exp'] = agg['EF'].apply(lambda x: weight_adjust(x, weight.loc[year]['icev'], 1.9))
    agg['EF_Ei5_exp'] = agg['EF_Ei5'].apply(lambda x: weight_adjust(x, weight.loc[year]['ev'], 1.9))
    agg['EF_ModelY_exp'] = agg['EF_ModelY'].apply(lambda x: weight_adjust(x, weight.loc[year]['ev'], 1.9))

In [24]:
columns = ['year', 'reg', 'a', 'EF', 'std']
EFs_year_trend = pd.DataFrame({}, columns=columns)

# calcualte weight-dependent EF
for year, agg in agg_links.items():
    for reg in ['zero', 'weak', 'strong']:
        for a in [1, 1.9]:
            # select assumption and EF key
            assumption = '_linear' if a == 1 else '_exp'
            if reg == 'zero':
                EF_key = 'EF'
            elif reg == 'weak':
                EF_key = 'EF_Ei5'
            else:
                EF_key = 'EF_ModelY'
            # calculate
            EF_key += assumption
            avg_EF = sum(agg[EF_key] * agg['trajCount']) / agg['trajCount'].sum()
            EF_std = agg.apply(lambda x: x['trajCount']*np.power((x[EF_key]-avg_EF), 2), axis=1).sum()
            EF_std /= agg['trajCount'].sum()
            EF_std = np.sqrt(EF_std)
            EFs_year_trend.loc[EFs_year_trend.shape[0]] =[year, reg, a, avg_EF, EF_std]

In [ ]:
# EFs_year_trend.to_csv("data/ev/EFs_year_trend.csv", index=False)

In [26]:
linear = EFs_year_trend[EFs_year_trend['a'] == 1]
zero_linear = linear[linear['reg'] == 'zero'].set_index('year')
weak_linear = linear[linear['reg'] == 'weak'].set_index('year')
strong_linear = linear[linear['reg'] == 'strong'].set_index('year')
((zero_linear['EF'] - weak_linear['EF']) / zero_linear['EF']).mean(), ((zero_linear['EF'] - strong_linear['EF']) / zero_linear['EF']).mean()

(0.6857197974905731, 0.916759841652949)

In [27]:
exp = EFs_year_trend[EFs_year_trend['a'] == 1.9]
zero_exp = exp[exp['reg'] == 'zero'].set_index('year')
weak_exp = exp[exp['reg'] == 'weak'].set_index('year')
strong_exp = exp[exp['reg'] == 'strong'].set_index('year')
((zero_exp['EF'] - weak_exp['EF']) / zero_exp['EF']).mean(), ((zero_exp['EF'] - strong_exp['EF']) / zero_exp['EF']).mean()

(0.7038998823575457, 0.9215750133723458)

### Consider penetration

In [28]:
# calculate avg EFs considering penetration
for year, agg in agg_links.items():
    pen = penetration.loc[year]['penetration']
    # linear assumption
    agg['EF_weak_pen_linear'] = agg.apply(lambda x: x['EF_linear']*(1-pen) + x['EF_Ei5_linear']*pen, axis=1)
    agg['EF_strong_pen_linear'] = agg.apply(lambda x: x['EF_linear']*(1-pen) + x['EF_ModelY_linear']*pen, axis=1)
    # exponetial assumption
    agg['EF_weak_pen_exp'] = agg.apply(lambda x: x['EF_exp']*(1-pen) + x['EF_Ei5_exp']*pen, axis=1)
    agg['EF_strong_pen_exp'] = agg.apply(lambda x: x['EF_exp']*(1-pen) + x['EF_ModelY_exp']*pen, axis=1)

In [29]:
columns = ['year', 'a', 'weak EF', 'strong EF', 'weak std', 'strong std']
avg_EFs_year_trend = pd.DataFrame({}, columns=columns)

# calcualte weight-dependent EF
for year, agg in agg_links.items():
    for a in [1, 1.9]:
        # select assumption and EF key
        assumption = '_linear' if a == 1 else '_exp'
        # calculate
        EF_key_weak = 'EF_weak_pen' + assumption
        EF_key_strong = 'EF_strong_pen' + assumption
        # weak
        avg_EF_weak = sum(agg[EF_key_weak] * agg['trajCount']) / agg['trajCount'].sum()
        EF_std_weak = agg.apply(lambda x: x['trajCount']*np.power((x[EF_key_weak]-avg_EF_weak), 2), axis=1).sum()
        EF_std_weak /= agg['trajCount'].sum()
        EF_std_weak = np.sqrt(EF_std_weak)
        # strong
        avg_EF_strong = sum(agg[EF_key_strong] * agg['trajCount']) / agg['trajCount'].sum()
        EF_std_strong = agg.apply(lambda x: x['trajCount']*np.power((x[EF_key_strong]-avg_EF_strong), 2), axis=1).sum()
        EF_std_strong /= agg['trajCount'].sum()
        EF_std_strong = np.sqrt(EF_std_strong)
        # save
        avg_EFs_year_trend.loc[avg_EFs_year_trend.shape[0]] =[year, a, avg_EF_weak, avg_EF_strong, EF_std_weak, EF_std_strong]

In [ ]:
# avg_EFs_year_trend.to_csv('data/ev/avg_EFs_year_trend.csv', index=False)

### penetration sensitivity

In [31]:
columns = ['year', 'pen', 'EF_linear', 'EF_exp']
EFs_year_pen_sensitivity = pd.DataFrame({}, columns=columns)

# calculate avg EFs considering penetration
for year, agg in agg_links.items():
    trajCount = agg['trajCount']
    for pen in np.arange(0., 0.5, 0.01):
        # linear assumption
        EF_weak_pen_linear = agg.apply(lambda x: x['EF_linear']*(1-pen) + x['EF_Ei5_linear']*pen, axis=1)
        EF_strong_pen_linear = agg.apply(lambda x: x['EF_linear']*(1-pen) + x['EF_ModelY_linear']*pen, axis=1)
        mean_EF_pen_linear = 0.5 * (
            sum(EF_weak_pen_linear * trajCount) / trajCount.sum() +
            sum(EF_strong_pen_linear * trajCount) / trajCount.sum()
        )
        # exponetial assumption
        EF_weak_pen_exp = agg.apply(lambda x: x['EF_exp']*(1-pen) + x['EF_Ei5_exp']*pen, axis=1)
        EF_strong_pen_exp = agg.apply(lambda x: x['EF_exp']*(1-pen) + x['EF_ModelY_exp']*pen, axis=1)
        mean_EF_pen_exp = 0.5 * (
            sum(EF_weak_pen_exp * trajCount) / trajCount.sum() +
            sum(EF_strong_pen_exp * trajCount) / trajCount.sum()
        )
        # save
        EFs_year_pen_sensitivity.loc[EFs_year_pen_sensitivity.shape[0]] = [year, pen, mean_EF_pen_linear, mean_EF_pen_exp]

In [32]:
EFs_year_pen_sensitivity.to_csv("data/ev/EFs_year_pen_sensitivity.csv", index=False)

### penetration and EV weight increasing rate

In [28]:
inc_rate = pd.DataFrame()
inc_rate['pen_inc_rate'] = penetration['penetration'] - penetration['penetration'].shift(1)
inc_rate['icev_weight_inc_rate'] = (weight['icev'] - weight['icev'].shift(1)) / weight['icev'].shift(1)
inc_rate['ev_weight_inc_rate'] = (weight['ev'] - weight['ev'].shift(1)) / weight['ev'].shift(1)
inc_rate

,pen_inc_rate,icev_weight_inc_rate,ev_weight_inc_rate
year,,,
2016,NaN,NaN,NaN
2017,0.001540,-0.000042,0.002412
2018,0.004248,0.001204,0.046451
2019,0.003355,0.002281,0.024192
2020,0.002855,0.005121,0.035624
2021,0.008425,0.003953,0.048099
2022,0.015132,0.004143,0.019382


In [29]:
# inc_rate.to_csv("data/ev/increasing_rate.csv")